# Selection Playbook

Notebook scaffold for ranking assets based on breakout heuristics, on-chain flows, and risk filters.

## TODO
- [ ] Load price, volume, and on-chain snapshots from QuantConnect and CoinGecko datasets.
- [ ] Engineer factor buckets (acceleration, relative volume, whale concentration, social buzz).
- [ ] Assemble composite selection scores, tier assets, and export candidate lists.


In [ ]:
# %pip install --quiet quantconnect-stubs
from datetime import datetime
from datetime import timedelta
from typing import Iterable

import pandas as pd

pd.options.display.max_columns = 20

try:
    from QuantConnect import Resolution  # type: ignore[import]
    from QuantConnect.Research import QuantBook  # type: ignore[import]
except ImportError as exc:  # pragma: no cover - requires QuantConnect runtime
    raise RuntimeError(
        "QuantConnect Research assemblies not found. Run inside a Lean Research environment."
    ) from exc

from research.scripts.data_loader import DataLoader, DataRequestSpec

qb = QuantBook()

def add_crypto_universe(tickers: Iterable[str], resolution: Resolution = Resolution.Hour):
    """Register crypto assets for the research universe and return their Symbol objects."""

    symbols = []
    for ticker in tickers:
        symbol = qb.AddCrypto(ticker, resolution).Symbol
        symbols.append(symbol)
    return symbols

UNIVERSE = ["BTCUSD", "ETHUSD", "SOLUSD"]
SYMBOLS = add_crypto_universe(UNIVERSE)

REQUEST_START = datetime(2024, 1, 1)
REQUEST_END = datetime.utcnow()

loader = DataLoader()
for symbol in SYMBOLS:
    loader.register(
        DataRequestSpec(
            symbol=symbol.Value,
            market=symbol.ID.Market,
            security_type="Crypto",
            resolution=Resolution.Hour.name,
            start=REQUEST_START.isoformat(),
            end=REQUEST_END.isoformat(),
        )
    )


In [ ]:
RUN_SAMPLE_HISTORY = False

if RUN_SAMPLE_HISTORY:
    lookback = timedelta(days=90)
    history = qb.History(SYMBOLS, lookback, Resolution.Hour)
    price_panel = (
        history.close.unstack(level=0)
        .tz_localize(None)
        .ffill()
    )
    price_panel.tail()
